In [ ]:
!pip install -U datasets evaluate transformers langchain langchain-openai faiss-cpu langchain-community llamaapi


from dotenv import load_dotenv
from openai import OpenAI
from google.colab import drive
from transformers import pipeline
from datasets import load_dataset, Dataset
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from collections import defaultdict
import json
import os
import torch
import re
import csv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: pyarrow
    Found existing

In [ ]:
HF_TOKEN =#insertyourkey


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load dataset
dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/role_aware_squad.json",
    split="train"
)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
#skip first thousand coz we used them for training and evaluating
def balance_role_dataset(dataset, per_role_count=500):

    role_data = defaultdict(lambda: {"answerable": [], "unanswerable": []})

    for i, example in enumerate(dataset):
        if i < 1000:
          continue
        for role in ["CUSTOMER"]:
            answerable = example["role_answerable"][role]
            entry = {
                "question": example["question"],
                "original_context": example["original_context"],
                "original_answer": example["original_answer"],
                "role_context": example["role_contexts"][role],
                "is_answerable": answerable,
                "role": role
            }
            category = "answerable" if answerable else "unanswerable"
            role_data[role][category].append(entry)

    # Sample balanced set
    balanced_examples = []
    for role in role_data:
        ans = role_data[role]["answerable"]
        unans = role_data[role]["unanswerable"]

        sampled = ans[:per_role_count] + unans[:per_role_count]
        balanced_examples.extend(sampled)

    return balanced_examples

In [ ]:
balanced_dataset = balance_role_dataset(dataset)

In [ ]:
#Build vectorstore
#texts = set([doc["original_context"] for doc in balanced_dataset])
#vectorstore = build_vectorstore(texts)


In [ ]:
def final_prompt(query, context):
    return f"""
    You are not allowed to use any outside knowledge, training data, or prior information. You must only answer based on the text provided below. If a detail (such as a name) is not explicitly written in the text, respond by saying "The information is not provided in the source."

    [BEGIN SOURCE TEXT]
    {context}
    [END SOURCE TEXT]

    Please answer the question
    Question: {query}
    """

In [ ]:
def is_actual_answer(response):

    response_clean = response.strip().lower()

    # List of phrases that indicate no answer was found
    denial_phrases = [
        "not provided in source",
        "not provided in the source",
        "not mentioned in source",
        "not mentioned in the source",
        "does not provide",
        "cannot be found",
        "information is not available",
    ]

    # Check if response matches any denial phrase
    for phrase in denial_phrases:
        if phrase in response_clean:
            return False

    return True

In [ ]:
#store as csv
output_path = '/content/drive/MyDrive/llm_rag_eval_heuristic_results_vanilla_1.csv'
# column names
fieldnames = [
    "index", "question", "context",
    "llm_answer", "llm_says_answerable",
    "original_answer"
]

# does file exist?
file_exists = os.path.isfile(output_path)

# open file
with open(output_path, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    if not file_exists:
        writer.writeheader()

In [ ]:
#process a simple rag

results = []
existing_indices = set()


index_counter = 0
for i in range(1000):
  index_counter += 1
  question = balanced_dataset[i]['question']
  context = balanced_dataset[i]['original_context']

  # Prompt + LLM
  prompt = final_prompt(question, context)

  client = OpenAI(
    base_url="https://gr8l7l65v3sk6xz2.us-east-1.aws.endpoints.huggingface.cloud/v1/",
    api_key=HF_TOKEN
  )

  r = client.chat.completions.create(
        model = "tgi",
        messages = [
          {
            "role": "user",
            "content": prompt
          }
        ],
        max_tokens=512
      )
  response = r.choices[0].message.content
  bool_answer = is_actual_answer(response)

  print(i)

  with open(output_path, mode='a', newline='', encoding='utf-8') as f:
          writer = csv.DictWriter(f, fieldnames=fieldnames)
          writer.writerow({
              "index": i,
              "question": question,
              "context": context,
              "llm_answer": response,
              "llm_says_answerable": bool_answer,
              "original_answer": balanced_dataset[i]['original_answer']
          })





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27